# The 8-puzzle

## Standard Formulation

In [1]:
# Including path to previous directory in built-in variable sys.path

import sys

sys.path.append('../')

In [2]:
# Importing the standard formulation

import numpy as np
from puzzle import formulations  # formulation returns the default
from collections import deque

In [3]:
# Class with a search problem

class Problem:
    def __init__(self, initial, objective, actions, result):
        self.initial = initial
        self.objective = objective
        self.actions = actions
        self.result = result  # defines the transition model

In [4]:
# Class with a tree node

class Node:
    def __init__(self, state, cost, parent, action):
        self.state = state
        self.cost = cost
        self.parent = parent
        self.action = action

    def __str__(self):  # representing the node
        return '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[0:2]]) + \
               '  >>>  ' + str(self.cost) + '\n\n' + \
               '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[2:]])

    @classmethod
    def child(cls, problem, parent, action):
        state = problem.result(parent.state, action)
    
        return cls(state, parent.cost + 1, parent, action)  # returning the child

    @property
    def solution(self):
        node = self;
        solution = []

        while node:
            solution.append(node)
            node = node.parent

        solution.reverse()

        return solution

In [5]:
# Creating new instances of the problem

grid1 = np.array([[3, 2, 7],
                  [4, 8, 1],
                  [6, 5, 0]])

grid2 = np.array([[3, 2, 7],
                  [4, 8, 0],
                  [6, 5, 1]])

grid3 = np.array([[3, 2, 0],
                  [4, 8, 7],
                  [6, 5, 1]])

puzzle_12_steps = Problem(Node(grid1, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_13_steps = Problem(Node(grid2, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_14_steps = Problem(Node(grid3, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

## Breadth-First Search

Efficient for paths with up to 12 steps:

In [6]:
# Constants for current search status

SEARCH_NOT_STARTED = 0
SEARCH_STARTED = 1
SEARCH_FAIL = 2
SEARCH_SUCCESS = 3

# Class that will implement breadth-first search

class BFS:
    def __init__(self, problem):
        self.problem = problem
        self.frontier = deque([])
        self.explored = deque([])
        self.situation = SEARCH_NOT_STARTED
        self.solution = []

    def step_search(self):
        # Root node initial check
        if self.situation == SEARCH_NOT_STARTED:  # only if the search did not fail or was successful
            if self.problem.objective(self.problem.initial.state):
                self.solution.append(self.problem.initial)
                self.situation = SEARCH_SUCCESS

                print('Root corresponds to the search objective!')
                return
            else:
                self.frontier.append(self.problem.initial)
                self.situation = SEARCH_STARTED  # indicates that the search has started

        # Checking if the search process failed
        if self.situation == SEARCH_FAIL:
            print("Search process failed!")
            return

        # Checking if the search was successful
        if self.situation == SEARCH_SUCCESS:
            print("Solution already found!")
            return
    
        if not all(self.frontier):  # empty border ends the search
            self.situation = SEARCH_FAIL
            return

        # Performing the search step
        node = self.frontier.popleft()

        self.explored.append(node.state)

        for action in self.problem.actions(node.state):
            child = Node.child(self.problem, node, action)
            
            if child not in self.frontier and not self.explored_node(child.state):
                if self.problem.objective(child.state):
                    self.solution = child.solution
                    self.situation = SEARCH_SUCCESS
                    return
                
                self.frontier.append(child)

    def search(self):
        # Loop that performs breadth-first search
        while self.situation != SEARCH_FAIL and self.situation != SEARCH_SUCCESS:
            self.step_search()

        if self.situation == SEARCH_FAIL:
            print("Search process failed!")
        else:
            print("Solution found: ")
            print(self.show_solution)
        
    def explored_node(self, state):
        for state_explored in self.explored:
            if np.all(state == state_explored):
                return True
            
        return False

    @property
    def show_solution(self):
        if self.situation == SEARCH_SUCCESS:
            return '\n'.join([node.__str__() for node in self.solution]) + \
                  f'\nCusto: {self.solution[-1].cost}'
    
        print("Solução ainda não foi encontrada!")  

    @property
    def show_frontier(self):
        return '#'*15 + '\n' + \
               '\n'.join([node.__str__() for node in self.frontier]) + \
               '\n' + '#'*15

Testing breadth-first search:

In [7]:
bfs = BFS(puzzle_12_steps)

In [8]:
%%time

bfs.search()  # checking the efficiency

Solution found: 
3  2  7

4  8  1  >>>  0

6  5  0
3  2  7

4  8  1  >>>  1

6  0  5
3  2  7

4  0  1  >>>  2

6  8  5
3  2  7

4  1  0  >>>  3

6  8  5
3  2  0

4  1  7  >>>  4

6  8  5
3  0  2

4  1  7  >>>  5

6  8  5
3  1  2

4  0  7  >>>  6

6  8  5
3  1  2

4  7  0  >>>  7

6  8  5
3  1  2

4  7  5  >>>  8

6  8  0
3  1  2

4  7  5  >>>  9

6  0  8
3  1  2

4  0  5  >>>  10

6  7  8
3  1  2

0  4  5  >>>  11

6  7  8
0  1  2

3  4  5  >>>  12

6  7  8
Custo: 12
CPU times: user 5.15 s, sys: 33.7 ms, total: 5.18 s
Wall time: 5.14 s


Breadth-first search via function:

In [9]:
def bfs(problem):
    node = problem.initial
    
    if problem.objective(node.state):
        return node.solution
    
    frontier = deque([node])
    explored = deque([])
    
    while True:
        if not all(frontier):  # empty border ends the search
            print("Search process failed!")
            return None

        # Performing the search step
        node = frontier.popleft()
        
        explored.append(node.state)
        
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            
            visited = False
            for state_explored in explored:
                if np.all(child.state == state_explored):
                    visited = True
                    break
            
            if child not in frontier and not visited:
                if problem.objective(child.state):
                    print("Solução encontrada!")
                    return child.solution
      
                frontier.append(child)

def show_solution(solution):
    print('\n'.join([node.__str__() for node in solution]) + \
         f'\nCusto: {solution[-1].cost}')

In [10]:
%%time

show_solution(bfs(puzzle_12_steps))  # checking the efficiency

Solução encontrada!
3  2  7

4  8  1  >>>  0

6  5  0
3  2  7

4  8  1  >>>  1

6  0  5
3  2  7

4  0  1  >>>  2

6  8  5
3  2  7

4  1  0  >>>  3

6  8  5
3  2  0

4  1  7  >>>  4

6  8  5
3  0  2

4  1  7  >>>  5

6  8  5
3  1  2

4  0  7  >>>  6

6  8  5
3  1  2

4  7  0  >>>  7

6  8  5
3  1  2

4  7  5  >>>  8

6  8  0
3  1  2

4  7  5  >>>  9

6  0  8
3  1  2

4  0  5  >>>  10

6  7  8
3  1  2

0  4  5  >>>  11

6  7  8
0  1  2

3  4  5  >>>  12

6  7  8
Custo: 12
CPU times: user 5.16 s, sys: 20 ms, total: 5.18 s
Wall time: 5.16 s


## Backtracking search

Efficient for paths with up to 14 steps:

In [11]:
reverse_state = {'l': 'r',
                 'r': 'l',
                 'd': 'u',
                 'u': 'd'}

def backtracking(problem, limit):
    solution = np.array([None for i in range(limit+1)])
    
    size = 0
    solution[size] = (problem.initial.state,
                      problem.actions(problem.initial.state))
    
    while not problem.objective(solution[size][0]):        
        if size == limit:
            size -= 1
            
            while not solution[size][1]:
                size -= 1
                
                if size == -1:
                    return None

        state = problem.result(solution[size][0], solution[size][1].pop())

        size += 1
        solution[size] = (state, problem.actions(state))
    
    return solution

Testing backtracking search:

In [12]:
%%time

solution = backtracking(puzzle_12_steps, 12)

len(solution)

CPU times: user 8.66 s, sys: 195 ms, total: 8.85 s
Wall time: 8.59 s


13

In [13]:
%%time

solution = backtracking(puzzle_13_steps, 13)

len(solution)

CPU times: user 8.23 s, sys: 279 ms, total: 8.51 s
Wall time: 8.18 s


14

In [14]:
%%time

solution = backtracking(puzzle_14_steps, 14)

len(solution)

CPU times: user 8.67 s, sys: 359 ms, total: 9.03 s
Wall time: 8.66 s


15